# Тематики
Пример отчета по расчету аудитории интернет-ресурсов (Сross Web) для выбранных тематик ресурсов и тематик медиа продуктов.

## Описание задачи

Посчитаем аудиторию в разбивке на ресурсы:
- данные по тематике ресурсов "Агрегаторы";
- данные по тематике медиа продуктов "Агрегаторы".

Общие параметры:
- Период: Январь 2022
- География: Россия 0+
- Население: 12+
- Тип пользования интернетом: ограничения нет, считаем по всем (Web Desktop, Web Mobile, App Mobile)

Статистики:
- Reach (reach)
- Reach% (reachPer)

# Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к API Cross Web и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
#import matplotlib.pyplot as plt
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.crossweb import tasks as cwt
from mediascope_api.crossweb import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с API Cross Web
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.CrossWebTask()
cats = cwc.CrossWebCats()

## Формируем задания

Необходимо сформировать два задания для API Cross Web:
- расчет аудитории ресурсов по тематике для ресурсов "Агрегаторы";
- расчет аудитории ресурсов по тематике для медиа продуктов "Агрегаторы".

### Общие параметры для заданий

Для начала зададим общие параметры

In [ ]:
# Задаем период
# Период указывается в виде списка ('Начало', 'Конец'). Можно указать несколько периодов
date_filter = [('2022-01-01', '2022-01-31')]

# Задаем фильтр по типам пользования интернетом
usetype_filter = [1,2,3]

### Получим ID тематик
Для построения отчета найдем все тематики ресурсов и тематики медиа продуктов, содержащие текст __"Агрегатор"__.

Для этого воспользуемся методами поиска в медиа справочнике. Работа с медиа справочником подробно описана в ноутбуке [catalogs](catalogs.ipynb). 

**ID тематик для ресурсов**


In [ ]:
cats.get_resource_theme(resource_theme='Агрегатор')

Таким образом, необходимые идентификаторы тематик для ресурсов следующие:

- **Агрегаторы товаров и услуг (c2c)**  crossMediaResourceThemeId = 39
- **Агрегаторы товаров и услуг: товарные маркетплейсы**  crossMediaResourceThemeId = 26

**ID тематик для медиа продуктов**

In [ ]:
cats.get_theme(theme='Агрегатор')

Таким образом, необходимые идентификаторы тематик для медиа продуктов следующие:

- **Агрегаторы товаров и услуг (c2c)**  crossMediaThemeId = 39
- **Агрегаторы товаров и услуг: товарные маркетплейсы**  crossMediaThemeId = 26

## Задания

Перейдем к формированию заданий.


### Задание №1. Расчет аудитории ресурсов по тематике для ресурсов "Агрегаторы"

In [ ]:
# Задаем название ресурса для отображения в DataFrame
project_name = 'Тематики для ресурсов'

# Задаем фильтр по географии, в нашем случае он не требуется
geo_filter = None

# Задаем фильтр по демографии, в нашем случае он не требуется
demo_filter = None

# Задаем фильтр тематикам ресурсов
mart_filter = 'crossMediaResourceThemeId in (39, 26)'

# Указываем список срезов, чтобы сформировать структуру расчета
slices = ["researchMonth", "crossMediaResourceName"]

# Указываем список статистик для расчета
statistics = ['reach', 'reachPer'] 

# Формируем задание для API Cross Web в формате JSON
task_json = mtask.build_task('media', project_name, date_filter, usetype_filter, geo_filter,
                                      demo_filter, mart_filter, slices, statistics)

# Отправляем задание на расчет и ждем выполнения
task_audience = mtask.wait_task(mtask.send_audience_task(task_json))

# Получаем результат
df_theme_resource = mtask.result2table(mtask.get_result(task_audience), project_name = project_name)
df_theme_resource

### Задание №2. Расчет аудитории ресурсов по тематике для медиа продуктов "Агрегаторы"

In [ ]:
# Задаем название ресурса для отображения в DataFrame
project_name = 'Тематики для медиа продуктов'

# Задаем фильтр по географии, в нашем случае он не требуется
geo_filter = None

# Задаем фильтр по демографии, в нашем случае он не требуется
demo_filter = None

# Задаем фильтр тематикам ресурсов
mart_filter = 'crossMediaThemeId in (39, 26)'

# Указываем список срезов, чтобы сформировать структуру расчета
slices = ["researchMonth", "crossMediaResourceName"]

# Указываем список статистик для расчета
statistics = ['reach', 'reachPer'] 

# Формируем задание для API Cross Web в формате JSON
task_json = mtask.build_task('media', project_name, date_filter, usetype_filter, geo_filter,
                                      demo_filter, mart_filter, slices, statistics)

# Отправляем задание на расчет и ждем выполнения
task_audience = mtask.wait_task(mtask.send_audience_task(task_json))

# Получаем результат
df_theme = mtask.result2table(mtask.get_result(task_audience), project_name = project_name)
df_theme

### Формирование итоговой таблицы

Следующим шагом соберем полученные результаты в один DataFrame

In [ ]:
# Объединим наши DataFrame'ы
df_result = pd.concat([df_theme_resource, df_theme])
df_result

Из полученного результата видно, что данные по одним и тем же ресурсам, посчитанные по одинаковым тематикам для ресурсов и тематикам для медиа продуктов, могут не совпадать.

Это объясняется тем, что при определении тематики для ресурсов для всего ресурса выбирается только одна наиболее подходящая тематика, даже если ресурс состоит из медиа продуктов, относящихся к различным тематикам. 

## Сохраняем в Excel

In [ ]:
writer = pd.ExcelWriter(mtask.get_excel_filename('audience-theme'))
df_info = mtask.get_report_info()
df_result.to_excel(writer, 'Report', index=False)
df_info.to_excel(writer, 'Info', index=False)
writer.save()